In [3]:
using PyPlot
using HDF5
using PyCall
using Statistics
mpl = pyimport("matplotlib")

PyObject <module 'matplotlib' from '/global/homes/k/kyuen2/.julia/conda/3/lib/python3.10/site-packages/matplotlib/__init__.py'>

In [2]:
#mode = ath2h5.DecompressionMode("Adiabatic",true,false)
Nx = Ny = Nz = 800
db = "/global/cscratch1/sd/kyuen2/AthenaMP/B_3miG/MP_MHD_800.out2.00013.athdf"
#Cubes = ath2h5.ConvertAthdf(db,Nx,Ny,Nz,mode)

"/global/cscratch1/sd/kyuen2/AthenaMP/B_3miG/MP_MHD_800.out2.00013.athdf"

In [4]:
f = h5open(db,"r");
prim = read(f,"prim");
LL   = read(f,"LogicalLocations");

In [5]:
function puzzleX(prim,LL,k,Nx,Ny,Nz)
	c=map(Float32,zeros((Nx,Ny,Nz)));
	#Nx,Ny,Nz = [maximum(LL[1,:]),maximum(LL[2,:]),maximum(LL[3,:])].*???
	dN = [size(prim)[1];size(prim)[2];size(prim)[3]];
	for i = 1:size(LL)[2]
		x1s, x2s, x3s = (LL[:,i].*dN).+1;
		x1f, x2f, x3f = (LL[:,i].+1).*dN;
		@views c[x1s:x1f, x2s:x2f, x3s:x3f] = prim[:,:,:,i,k];
	end
	c
end

puzzleX (generic function with 1 method)

In [ ]:
B = read(f,"B");
ib = puzzleX(B,LL,1,Nx,Ny,Nz);
jb = puzzleX(B,LL,2,Nx,Ny,Nz);
kb = puzzleX(B,LL,3,Nx,Ny,Nz);

In [6]:
d = puzzleX(prim,LL,1,Nx,Ny,Nz);
p = puzzleX(prim,LL,2,Nx,Ny,Nz);
#iv= puzzleX(prim,LL,3,Nx,Ny,Nz);
#jv= puzzleX(prim,LL,4,Nx,Ny,Nz);
#kv= puzzleX(prim,LL,5,Nx,Ny,Nz);

In [7]:
prim = 0
GC.gc();

In [30]:
cs = sqrt.(γ*p./d) 
maximum(cs)

7.191000410314663

In [35]:
dx = 200/Nx;
t_cs = dx/maximum(cs)*0.3

0.010429703201298823

In [27]:
function CoolingTime_T(d::Array{DT,3},p::Array{DT,3}) where DT
    unit_length_in_cm_  = 3.086e+18;
    unit_vel_in_cms_    = 1.0e5;
    unit_density_in_nH_ = 1;
    γ = 5/3;
    unit_E_in_cgs_ = 1.67e-24 * 1.4 * unit_density_in_nH_* unit_vel_in_cms_ * unit_vel_in_cms_;
    unit_time_in_s_ = unit_length_in_cm_/unit_vel_in_cms_;
    nx,ny,nz = size(d);
    mindt = 1e99
    for k = 1:nz::Int, j = 1:ny::Int
        @simd for i = 1:nx::Int
        
        nH_ = d[i,j,k];
        ED  = p[i,j,k]/(γ-1); 
        E_ergs = ED * unit_E_in_cgs_ / nH_;
        T  =  E_ergs / (1.5*1.381e-16);
            
            
        Heating = 2e-26;
        Cooling = 2e-26*nH_*(1e7*exp(-1.184e5/(T+ 1e3)) + 1.4e-2*sqrt(T)*exp(-92/T)); 
        dEdt = Heating;
        dEdt =  T > 50 ? dEdt : dEdt - Cooling;
        E_ergs = T*(1.5*1.381e-16);
        dt = abs(0.3*E_ergs/dEdt/unit_time_in_s_);
        mindt = mindt > dt ? dt : mindt;
            
        end
    end
    return mindt
end

CoolingTime_T (generic function with 1 method)

In [28]:
CoolingTime_T(d,p)

0.00118662005247984

In [29]:
T = 8000
sqrt(T/170)

6.859943405700354

In [21]:
mean(cs)

3.80118004273731

In [20]:
mean(iv),mean(jv),mean(kv)

(-0.05193824f0, 0.033339016f0, -0.0098565f0)

In [16]:
std(iv),std(jv),std(kv)

(1.1483554f0, 1.1142577f0, 1.396002f0)

In [32]:
unit_length_in_cm_  = 3.086e+18;
unit_vel_in_cms_    = 1.0e5;
unit_density_in_nH_ = 1;
γ = 5/3;
unit_E_in_cgs_ = 1.67e-24 * 1.4 * unit_density_in_nH_* unit_vel_in_cms_ * unit_vel_in_cms_;
unit_time_in_s_ = unit_length_in_cm_/unit_vel_in_cms_;
T  =  p./(γ-1) * unit_E_in_cgs_ ./ d./ (1.5*1.381e-16);

In [ ]:
#Phase diagram
CoolRate(ρ::Number,T::Array,Γ::Number) = ρ^2*Γ*(1e7*exp.(-[1.184e5]./(T.+1e3))+1.4e-2*sqrt.(T).*exp.(-[92]./T));
TT = [10].^collect(-2:0.0001:5.0);
G = CoolRate(1,TT,1);
n  = G.^-1;
ndot = n*1;

plt[:hist2d](log10.(d)[:],log10.(170*p[:]),bins=300,cmap="jet",norm=mpl.colors.LogNorm())
plot(log10.(ndot),log10.(ndot.*TT),"k--")
xlim(-1,2.75)
ylim(2.5,4.3)#ylim(0,5.5)

xlabel(L"log_{10}(n) [cm^{-3}]",fontsize=15)
ylabel(L"log_{10}(P/k_b) ( K cm^{-3})",fontsize=15)

In [8]:
function imshow_with_sd(A;cmap="jet")
    m,σ = mean(A[.~isnan.(A)]),std(A[.~isnan.(A)]);
    imshow(A,vmin=m-2σ,vmax=m+2σ,cmap=cmap);colorbar()
end

figure(figsize=(21,9))
subplot(131);title(L"\rho")
imshow_with_sd(log10.(d[:,400,:]');cmap="jet")
subplot(132);title(L"p")
imshow_with_sd(log10.(p[:,400,:]');cmap="jet")
subplot(133);title(L"T")
imshow_with_sd(log10.(T[:,400,:]');cmap="jet")

LoadError: UndefVarError: T not defined

In [98]:
4352/17/16/16

1.0

In [103]:
816/16,816/17

(51.0, 48.0)